Part 1

In [47]:
# Import Libs

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [48]:
# Scrape Data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
scraped_data = BeautifulSoup(requests.get(url).text, 'lxml')

In [49]:
# Turn scraped data table into primitive Dataframe

cols = ['PostalCode', 'Borough', 'Neighbourhood']
neighbour_df = pd.DataFrame(columns=cols)

j = 0
for i in scraped_data.find('table').find_all('tr'):
    if (len(i.find_all('td')) > 0):
        PC = i.find_all('td')[0].text.strip()
        B = i.find_all('td')[1].text.strip()
        N = i.find_all('td')[2].text.strip()
        neighbour_df.loc[j] = [i.find_all('td')[0].text.strip(), i.find_all('td')[1].text.strip(), i.find_all('td')[2].text.strip()]
        j = j + 1
neighbour_df

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [53]:
# Clean Dataframe

neighbour_df = neighbour_df[neighbour_df.Borough != "Not assigned"].reset_index(drop=True)
neighbour_df = neighbour_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
neighbour_df.columns = ['PostalCode', 'Borough', 'Neighbourhood']
for i, j in neighbour_df.iterrows():
    if j['Neighbourhood'] == 'Not assigned':
        j['Neighbourhood'] = j['Borough']
print(neighbour_df)
neighbour_df.shape

    PostalCode      Borough                                      Neighbourhood
0          M1B  Scarborough                                     Malvern, Rouge
1          M1C  Scarborough             Rouge Hill, Port Union, Highland Creek
2          M1E  Scarborough                  Guildwood, Morningside, West Hill
3          M1G  Scarborough                                             Woburn
4          M1H  Scarborough                                          Cedarbrae
..         ...          ...                                                ...
98         M9N         York                                             Weston
99         M9P    Etobicoke                                          Westmount
100        M9R    Etobicoke  Kingsview Village, St. Phillips, Martin Grove ...
101        M9V    Etobicoke  South Steeles, Silverstone, Humbergate, Jamest...
102        M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


(103, 3)

Part 2

In [60]:
#Load Latitude and Longitude data 
gd = pd.read_csv('http://cocl.us/Geospatial_data')

In [61]:
# Merge with Neighbourhood Data
gd.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geo_neighbour_df = pd.merge(neighbour_df, gd, on="PostalCode", how='left')
geo_neighbour_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
